# How to create a simulation

This tutorial demonstrates how to create a simulation.

## What is a simulation?

A simulation contains selected sensors, sources to model ray-trace in space.

In [1]:
from pathlib import Path

from ansys.speos.core import GeoRef, Project, Speos
from ansys.speos.core.simulation import SimulationInteractive, SimulationInverse

# If using docker container
tests_data_path = Path("/app") / "assets"
# If using local server
# tests_data_path = Path().resolve().parent.parent / "tests" / "assets"

## Create connection with speos rpc server

In [2]:
speos = Speos(host="localhost", port=50098)

## Create Project

Create a new project first.

The only way to create a simulation is to create it from a project.

In [3]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


## Prepare prerequisites

Create the necessary elements for a simulation: Sensor, source, root part, optical property are prerequisites.

### Prepare the root part

In [4]:
root_part = p.create_root_part()
root_part.create_body(name="Body.1").create_face(name="Face.1").set_vertices(
    [0, 1, 2, 0, 2, 2, 1, 2, 2]
).set_facets([0, 1, 2]).set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
root_part.commit()

### Prepare an optical property

In [5]:
opt_prop = p.create_optical_property("Material.1")
opt_prop.set_volume_opaque().set_surface_mirror()  # vop as opaque and sop as mirror
# Choose the geometry for this optical property : Body.1
opt_prop.set_geometries(geometries=[GeoRef.from_native_link(geopath="Body.1")])
opt_prop.commit()

### Prepare an irradiance sensor

In [6]:
sensor1 = p.create_sensor(name="Irradiance.1")
sensor1.set_type_colorimetric()  # colorimetric or spectral so that the sensor can be used both in direct and inverse simulation
sensor1.commit()

### Prepare a surface source

In [7]:
source1 = p.create_source(name="Surface.1")
source1.set_exitance_constant(geometries=[(GeoRef.from_native_link(geopath="Body.1/Face.1"), True)])
source1.set_spectrum().set_blackbody()  # blackbody so that the source can be used both in direct and inverse simulation
source1.commit()

## Create a simulation

In [8]:
simulation1 = p.create_simulation(name="Simulation.1")
simulation1.set_sensor_paths(["Irradiance.1"]).set_source_paths(["Surface.1"])
print(simulation1)

simulation1.commit()
print(simulation1)

local: {
    "name": "Simulation.1",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "metadata": {},
    "simulation_guid": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_path": "Simulation.1",
        "job_type": "CPU"
    }
}
{
    "name": "Simulation.1",
    "metadata": {
        "Uniqu

## Set simulation characteristics

Simulation is defined with the same default values as the GUI speos.

If the user would like to modify the simulation characteristics,
it is possible to do so by setting the simulation characteristics as below.

In [9]:
simulation2_direct = p.create_simulation(name="Simulation.2")

simulation2_direct.set_ambient_material_file_uri(
    uri=str(tests_data_path / "AIR.material")
).set_colorimetric_standard_CIE_1964().set_weight_none().set_geom_distance_tolerance(
    0.01
).set_max_impact(200).set_dispersion(False)
simulation2_direct.set_sensor_paths(["Irradiance.1"]).set_source_paths(["Surface.1"]).commit()
print(simulation2_direct)

{
    "name": "Simulation.2",
    "metadata": {
        "UniqueId": "2fbacbca-46a5-400d-8b69-290e18b8f142"
    },
    "simulation_guid": "884b468c-53ed-4888-919e-c36ef3fe8829",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 200,
            "colorimetric_standard": "CIE_1964",
            "ambient_material_uri": "/app/assets/AIR.material",
            "dispersion": false,
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.2",
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_path": "Simulation.2",
        "job_type": "CPU"
    }
}


## Read information

Read simulation information

In [10]:
print(simulation1)

{
    "name": "Simulation.1",
    "metadata": {
        "UniqueId": "8b0da080-93b3-4f7e-b3f6-954466ae6e61"
    },
    "simulation_guid": "a579b8c2-9874-4ecb-847f-8efa07312d98",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_path": "Simulation.1",
   

Read project information

In [11]:
print(p)

{
    "part_guid": "59cff296-36f6-45fd-adf8-0d685439d439",
    "sources": [
        {
            "name": "Surface.1",
            "metadata": {
                "UniqueId": "d0fed246-19b5-4035-9d83-2a212da2af7f"
            },
            "source_guid": "dae3a2ab-71bb-4b3b-ad5f-36ea87d49161",
            "description": "",
            "source": {
                "name": "Surface.1",
                "surface": {
                    "luminous_flux": {
                        "luminous_value": 683.0
                    },
                    "intensity_guid": "b0e2a078-5a7f-4121-bb02-fb524cfa8762",
                    "exitance_constant": {
                        "geo_paths": [
                            {
                                "geo_path": "Body.1/Face.1",
                                "reverse_normal": true
                            }
                        ]
                    },
                    "spectrum_guid": "75fdae1b-e110-47a5-93c6-fb3022a6d1d0",
             

## Update simulation settings

If you are manipulating a simulation already committed, remember to commit your changes.

If you don't, you will still only watch what is committed on the server.

In [12]:
simulation1.set_ambient_material_file_uri(uri=str(tests_data_path / "AIR.material"))
simulation1.commit()
print(simulation1)

{
    "name": "Simulation.1",
    "metadata": {
        "UniqueId": "8b0da080-93b3-4f7e-b3f6-954466ae6e61"
    },
    "simulation_guid": "a579b8c2-9874-4ecb-847f-8efa07312d98",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "ambient_material_uri": "/app/assets/AIR.material",
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_pa

## Reset

Possibility to reset local values from the one available in the server.

In [13]:
simulation1.set_max_impact(1000)  # adjust max impact but no commit
simulation1.reset()  # reset -> this will apply the server value to the local value
simulation1.delete()  # delete (to display the local value with the below print)
print(simulation1)

local: {
    "name": "Simulation.1",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "metadata": {},
    "simulation_guid": "",
    "simulation": {
        "direct_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "dispersion": true,
            "ambient_material_uri": "/app/assets/AIR.material",
            "colorimetric_standard": "CIE_1931",
            "fast_transmission_gathering": false,
            "stop_condition_rays_number": "200000",
            "automatic_save_frequency": 1800
        },
        "name": "Simulation.1",
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_path": "Simulation.1",
        "job_type": "CPU"
    }
}


## Other simulation examples

### Inverse simulation

In [14]:
simulation3 = p.create_simulation(name="Simulation.3", feature_type=SimulationInverse)
simulation3.set_sensor_paths(sensor_paths=["Irradiance.1"]).set_source_paths(
    source_paths=["Surface.1"]
).commit()
print(simulation3)

{
    "name": "Simulation.3",
    "metadata": {
        "UniqueId": "ba94f095-8166-4143-a5fc-eb6b6644e57d"
    },
    "simulation_guid": "eda6a510-619e-4a89-a388-6b6354485257",
    "sensor_paths": [
        "Irradiance.1"
    ],
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "simulation": {
        "inverse_mc_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "number_of_gathering_rays_per_source": 1,
            "colorimetric_standard": "CIE_1931",
            "dispersion": false,
            "splitting": false,
            "maximum_gathering_error": 0,
            "maximum_gathering_error_percentage": 0.0,
            "fast_transmission_gathering": false,
            "ambient_material_uri": "",
            "optimized_propagation_none": {
                "stop_condition_passes_number": 5
            },
     

### Interactive simulation

In [15]:
simulation4 = p.create_simulation(name="Simulation.4", feature_type=SimulationInteractive)
simulation4.set_source_paths(source_paths=["Surface.1"]).commit()
print(simulation4)

{
    "name": "Simulation.4",
    "metadata": {
        "UniqueId": "a7bb803f-ec82-4288-8768-e45198578afe"
    },
    "simulation_guid": "fd2f73b3-5b09-4f1d-9727-8aa94fb20217",
    "source_paths": [
        "Surface.1"
    ],
    "description": "",
    "sensor_paths": [],
    "simulation": {
        "name": "Simulation.4",
        "interactive_simulation_template": {
            "geom_distance_tolerance": 0.01,
            "max_impact": 100,
            "weight": {
                "minimum_energy_percentage": 0.005
            },
            "colorimetric_standard": "CIE_1931",
            "ambient_material_uri": "",
            "rays_number_per_sources": [],
            "light_expert": false,
            "impact_report": false
        },
        "description": "",
        "metadata": {},
        "scene_guid": "26053d74-d329-428d-9054-7e08d9d5fd85",
        "simulation_path": "Simulation.4",
        "job_type": "CPU"
    }
}
